In [2]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain import PromptTemplate, OpenAI, LLMChain

llm = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio", 
                        temperature=0.8, model="mistralai_mistral-7b-instruct-v0.2")

In [4]:
template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}
"""

llm = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio", 
                temperature=0.8, model="mistralai_mistral-7b-instruct-v0.2")
prompt_template = PromptTemplate.from_template(template=template)
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.predict(input="I ordered Pizza Salami and it was awesome!")

'```json\n{\n  "sentiment": "positive",\n  "subject": "Pizza"\n}\n```'

## Sequentials Chains
Sometimes you want to pass the output from one model to a another model. This can be done with different SequentialsChains

In [5]:
response_template = """
You are a helpful bot that creates a 'thank you' reponse text. 
If customers are unsatisfied, offer them a real world assitant to talk to. 
You will get a sentiment and subject as into and evaluate. 

text: {input}
"""
review_template = PromptTemplate(input_variables=["input"], template=response_template)
review_chain = LLMChain(llm=llm, prompt=review_template)

In [6]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[chain, review_chain], verbose=True)

overall_chain.run(input="I ordered Pizza Salami and was aweful!")

c:\Users\InteractiveMachine\Desktop\Santiago\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
```json
{
  "sentiment": "negative",
  "subject": "pizza"
}
```

```text
I'm sorry that you weren't satisfied with the pizza. I understand how disappointing that can be. Could you please provide me with more details about your experience, such as the name of the pizzeria or your order number? I will do my best to help you find a resolution. In the meantime, if you prefer to speak with someone in person, I can connect you with a real world assistant who can assist you further.
```

> Finished chain.


"\n```text\nI'm sorry that you weren't satisfied with the pizza. I understand how disappointing that can be. Could you please provide me with more details about your experience, such as the name of the pizzeria or your order number? I will do my best to help you find a resolution. In the meantime, if you prefer to speak with someone in person, I can connect you with a real world assistant who can assist you further.\n```"

Chains can be more complex and not all sequential chains will be as simple as passing a single string as an argument and getting a single string as output for all steps in the chain

In [7]:
from langchain.chains import SequentialChain

# This is an LLMChain to write a review given a dish name and the experience.
prompt_review = PromptTemplate.from_template(
    template="You ordered {dish_name} and your experience was {experience}. Write a review: "
)
chain_review = LLMChain(llm=llm, prompt=prompt_review, output_key="review")

# This is an LLMChain to write a follow-up comment given the restaurant review.
prompt_comment = PromptTemplate.from_template(
    template="Given the restaurant review: {review}, write a follow-up comment: "
)
chain_comment = LLMChain(llm=llm, prompt=prompt_comment, output_key="comment")

# This is an LLMChain to summarize a review.
prompt_summary = PromptTemplate.from_template(
    template="Summarise the review in one short sentence: \n\n {review}"
)
chain_summary = LLMChain(llm=llm, prompt=prompt_summary, output_key="summary")

# This is an LLMChain to translate a summary into German.
prompt_translation = PromptTemplate.from_template(
    template="Translate the summary to german: \n\n {summary}"
)
chain_translation = LLMChain(
    llm=llm, prompt=prompt_translation, output_key="german_translation"
)

overall_chain = SequentialChain(
    chains=[chain_review, chain_comment, chain_summary, chain_translation],
    input_variables=["dish_name", "experience"],
    output_variables=["review", "comment", "summary", "german_translation"],
)

overall_chain({"dish_name": "Pizza Salami", "experience": "It was awful!"})

c:\Users\InteractiveMachine\Desktop\Santiago\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'dish_name': 'Pizza Salami',
 'experience': 'It was awful!',
 'review': "Title: Disappointing Experience with Pizza Salami\nReview: I recently ordered a Pizza Salami from [Company Name], but unfortunately my dining experience left me greatly disappointed. The pizza arrived cold, the crust was overcooked and tough, and the salami topping was undercooked and lacked flavor. I had high hopes for this meal, but it failed to meet my expectations in multiple ways. I would suggest improving the quality control of your pizzas to ensure that each one is cooked evenly and fully flavored. For now, I won't be ordering from [Company Name] again.",
 'comment': "Dear Valued Customer,\nWe are deeply sorry to hear about your recent disappointment with the Pizza Salami you ordered from us. We take all feedback seriously and strive to provide our customers with the best possible dining experience. Your comments have been shared with our team, and we will be conducting a thorough investigation into this i

Instead of chaining multiple chains together we can also use an LLM to decide which follow up chain is being used

In [8]:
from langchain.llms import OpenAI
from langchain.chains.router import MultiPromptChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

positive_template = """You are an AI that focuses on the positive side of things. \
Whenever you analyze a text, you look for the positive aspects and highlight them. \
Here is the text:
{input}"""

neutral_template = """You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, \
not favoring any positive or negative aspects. Here is the text:
{input}"""

negative_template = """You are an AI that is designed to find the negative aspects in a text. \
You analyze a text and show the potential downsides. Here is the text:
{input}"""

In [10]:
prompt_infos = [
    {
        "name": "positive",
        "description": "Good for analyzing positive sentiments",
        "prompt_template": positive_template,
    },
    {
        "name": "neutral",
        "description": "Good for analyzing neutral sentiments",
        "prompt_template": neutral_template,
    },
    {
        "name": "negative",
        "description": "Good for analyzing negative sentiments",
        "prompt_template": negative_template,
    },
]


destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
destination_chains

{'positive': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='You are an AI that focuses on the positive side of things. Whenever you analyze a text, you look for the positive aspects and highlight them. Here is the text:\n{input}'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002A388F53AF0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002A3890A91E0>, model_name='mistralai_mistral-7b-instruct-v0.2', temperature=0.8, openai_api_key='lm-studio', openai_api_base='http://localhost:1234/v1', openai_proxy='')),
 'neutral': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, not favoring any positive or negative aspects. Here is the text:\n{input}'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002A388F53AF0>, async_client=<openai.resources.chat.compl

In [11]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=destination_chains["neutral"],
    verbose=True,
)

chain.run("I ordered Pizza Salami for 9.99$ and it was awesome!")



> Entering new MultiPromptChain chain...
positive: {'input': 'I ordered Pizza Salami for 9.99$ and it was awesome!'}
> Finished chain.


'Absolutely, the Pizza Salami that you ordered for $9.99 was an amazing experience! The taste and satisfaction derived from it were truly delightful.'